In [1]:
# We import libraries that we will use throughout the code
import numpy as np # Array processing for numbers, strings, records, and objects
import collections # The librairy "collections" allows to import lists, dictionaries and sets


class Graph:# We create a class "Graph" that contains the following functions. Then we can call this class in the code
            # when needed. 
    def __init__(self): # The first function that we define inside the class "Graph" is "__init__". It initializes 
                        # the objects (variables) of the graph: nodes, edges, and distances. 
        self.nodes = set() # We create the variable "nodes" that is a set (empty at first)
        self.edges = collections.defaultdict(list)# We create the variable "edges" that is a dictionary. A dictionary 
                                                  # is a unordered list of keys associated to values.  We use the 
                                                  # dictionary "defaultdict" from "collections" that creates an empty 
                                                  # list and allows to append elements to it without having to 
                                                  #initialise each empty list.
        self.distances = {} # We create the variable "distances" that is also a dictionary. This dictionary is empty 
                            # at first.

    def add_node(self, value): # After having initialized the variables, we define the function "add_node". This 
                               # function corresponds to the first action that the class can do: adding a node to the 
                               # graph. And to add a node, we need a value. The value of a node is the accumulation of 
                               # the distances of the edges leading to this node.
        self.nodes.add(value) # To add the value into the set of nodes "self.nodes".

    def add_edge(self, from_node, to_node, distance): # This function is used to add an edge to the graph. To add an 
                                                      # edge, we need the node of origin of the edge: "from_node", its 
                                                      # destination: "to_node" and its weight: "distance" 
        self.edges[from_node].append(to_node) # We assign the position "to_node" (a  value) to the position 
                                              # "from_node" (a key).
        self.distances[(from_node, to_node)] = distance # to assign a distance from a node ("from_node") to 
                                                        # another ("to_node").

def dijkstra(graph, initial): # We define the Dijsktra function that will need a graph (that we will generate after) 
                              # and a source: "initial".
    m = {initial: 0} # m contains the nodes that have an estimated distance, hence that have been visited. 
    predecessor = {} 

    non_visited = set(graph.nodes)

# While loop
    while non_visited: # While some nodes are not visited,
        min_node = None # there is no minimum node yet. "min_node" is the node with the smallest value, in other words,
                        # it is the node that minimizes the distance from initial.

# the algorithm visits every node one by one.        
        for node in non_visited: # For every node in the graph
            if node in m: # if the node has an estimated distance 
                if min_node is None: # and if there is no min_node : if there is no node that has a value yet
                    min_node = node # then the new node becomes the min_node
                elif m[node] < m[min_node]: # however, if there is already a min_node, we check if the distance 
                                            # travelled to get to the new node is inferior to the one of the min_node.
                    min_node = node # and if it is the case, then the new node becomes the min_node

        if min_node is None:
            break # once every node has been visited, then there will be no more new min_node, and the "while" is over.

        non_visited.remove(min_node) # once min_node has been visited, it is removed from the set of the non visited 
                                    # nodes.

        current_weight = m[min_node] # the variable "current_weight" is distance travelled to get to min_node.

        for neighbor in graph.edges[min_node]: # for the edges that departs from the min_node, that we call "neighbor" 
                                               # of min_node

            weight = current_weight + graph.distances[(min_node, neighbor)] # the variable "weight" is equal to the  
                                                                            # sum of "current_weight" and the distance 
                                                                            # of the edge that goes from "min_node" to 
                                                                            # "neighbor".

            if neighbor not in m or weight < m[neighbor]: # if the neighbor does not have a value yet (doesn't have 
                                                          # the value of the edge that goes from min_node to neighbor) 
                                                          # OR if the weight of the distance when passing by min_node 
                                                          # is inferior to the weight we had before
                m[neighbor] = weight # the weight is updated
                predecessor[neighbor] = min_node # the predecessor is updated and becomes min_node

    return m, predecessor

# this cell uses the code found on https://gist.github.com/econchick/4666413, but with changes in variables' names. 

Then we write the algorithm that solves our business problem.

In [2]:
def find_optimal_intermediary(dep,arr,intermediaries,graph):
    '''
    This function will, given a graph, a departure node, 
    an arrival node find the best intermediary point to 
    pass by in order to minimize the total path length.

    @params
    :dep: the departure point (string)
    :arr: the arrival point (string)
    :intermediaries: the list of possible intermediary points (list of strings))
    :graph: the graph
    '''

    # We first call the function dijkstra to compute the distance from dep to all nodes in the graph
    m_dep, predecessor_dep = dijkstra(graph, dep)

    # Then for each of the intermediary points we compute 
    # the distance from the intermediary point to any other in the graph
    
    dijkstra_results = {} # We create the dictionary "dijkstra_results empty at first.
    for inter_point in intermediaries: #For an intermediary point in the lists of all intermediary points,
                                       # we store the results in a dictionary to be able to use it afterwards
        dijkstra_results[inter_point], _ = dijkstra(graph, inter_point) # To each key "inter_point" in the dictionary 
                                                                        # "dijkstra_results", we assign the value of
                                                                        # the output m. 

        
    # we have all the necessary information to compute the length of each path
    
    no_inter_dist = m_dep[arr] # distance between dep and arr without going through an inter point
    print("Without going through any intermediary points d = {}".format(no_inter_dist))
    inter_distances = [] # stores the distance of the path going through each interpoint
  
    for inter_point in intermediaries:
        dep_inter = m_dep[inter_point] # distance from departure to intermediary point
        inter_arr = dijkstra_results[inter_point][arr] # distance from intermediary point to arrival
        total_dist = dep_inter + inter_arr # the total distance equals the sum of the two previous distances
        inter_distances.append(total_dist) # We append the total_dist to the array "inter_distances"

    # we find the minimun distance
    optimal_inter = intermediaries[np.argmin(inter_distances)] # Returns the indices of the minimum values along all 
                                                               # intermediaries.
    optimal_dist = min(inter_distances) # Returns the minimum distance 

    print("Synthesis of the distance through each intermediary point:")
    for i,inter_point in enumerate(intermediaries):
        print("\tThrough {}, d = {}".format(inter_point,inter_distances[i]))

    print("The optimal intermediary point is {} (increasing the distance between '{}' and '{}' of {} distance units).".format(optimal_inter,dep,arr,optimal_dist-no_inter_dist))

In [3]:
# In order to test the program, we create a graph as an example
graph = Graph() # We call the class "Graph" to generate the graph. What the class can output will be stored in 
                # the variable "graph".

# Then we add the different nodes
graph.add_node('A') # (departure)
graph.add_node('B')
graph.add_node('C')
graph.add_node('D')
graph.add_node('E')
graph.add_node('F')
graph.add_node('G')
graph.add_node('H') # (arrival)
graph.add_node('I')

# Finally we create the different links between the edges
graph.add_edge('A','C',1)
graph.add_edge('A','B',2)
graph.add_edge('C','D',2)
graph.add_edge('D','B',3)
graph.add_edge('C','E',1)
graph.add_edge('D','E',2)
graph.add_edge('D','F',1)
graph.add_edge('B','F',6)
graph.add_edge('B','G',1)
graph.add_edge('E','H',1)
graph.add_edge('F','H',1)
graph.add_edge('G','H',3)
graph.add_edge('B','I',3)
graph.add_edge('I','',3)

# Now we will give more meaningful names to our nodes in order to use it for our business case
departure = 'A'
arrival = 'H'
inter1 = 'E'
inter2 = 'B'
inter3 = 'F'
intermediaries = [inter1,inter2,inter3]

# Now we can run our algorithm
find_optimal_intermediary(departure,arrival,intermediaries,graph)

Without going through any intermediary points d = 3
Synthesis of the distance through each intermediary point:
	Through E, d = 3
	Through B, d = 6
	Through F, d = 5
The optimal intermediary point is E (increasing the distance between 'A' and 'H' of 0 distance units).


In [4]:
# In order to test the program, we create a graph as an example
graph1 = Graph()

# Then we add the different nodes
graph1.add_node('A') # (departure)
graph1.add_node('B')
graph1.add_node('C')
graph1.add_node('D')
graph1.add_node('E')
graph1.add_node('F')
graph1.add_node('G')
graph1.add_node('H')# (arrival)
graph1.add_node('I')
graph1.add_node('J')
graph1.add_node('K')
graph1.add_node('L')
graph1.add_node('M')
graph1.add_node('N')

# Finally we create the different links between the edges
graph1.add_edge('A','C',1)
graph1.add_edge('A','B',2)
graph1.add_edge('C','D',2)
graph1.add_edge('D','B',3)
graph1.add_edge('C','E',1)
graph1.add_edge('D','E',2)
graph1.add_edge('D','F',1)
graph1.add_edge('B','F',6)
graph1.add_edge('B','G',1)
graph1.add_edge('E','H',1)
graph1.add_edge('F','H',1)
graph1.add_edge('G','H',3)
graph1.add_edge('B','J',2)
graph1.add_edge('J','F',2)
graph1.add_edge('E','I',2)
graph1.add_edge('I','F',1)
graph1.add_edge('K','F',6)
graph1.add_edge('L','H',5)
graph1.add_edge('A','L',3)
graph1.add_edge('B','M',1)
graph1.add_edge('M','E',6)
graph1.add_edge('M','N',3)
graph1.add_edge('N','H',1)


# Now we will give more meaningful names to our nodes in order to use it for our business case
departure = 'A'
arrival = 'H'
inter1 = 'E'
inter2 = 'B'
inter3 = 'F'
intermediaries = [inter1,inter2,inter3]

find_optimal_intermediary(departure,arrival,intermediaries,graph1)

Without going through any intermediary points d = 3
Synthesis of the distance through each intermediary point:
	Through E, d = 3
	Through B, d = 6
	Through F, d = 5
The optimal intermediary point is E (increasing the distance between 'A' and 'H' of 0 distance units).


In [5]:
# In order to test the program, we create a graph as an example
graph2 = Graph()

# Then we add the different nodes
graph2.add_node('A') # (departure)
graph2.add_node('B')
graph2.add_node('C')
graph2.add_node('D')
graph2.add_node('E')
graph2.add_node('F')
graph2.add_node('G')
graph2.add_node('H')# (arrival)
graph2.add_node('I')
graph2.add_node('J')
graph2.add_node('K')
graph2.add_node('L')
graph2.add_node('M')
graph2.add_node('N')
graph2.add_node('O')
graph2.add_node('P')
graph2.add_node('Q')
graph2.add_node('R')

# Finally we create the different links between the edges
graph2.add_edge('A','C',1)
graph2.add_edge('A','B',2)
graph2.add_edge('C','D',2)
graph2.add_edge('D','B',3)
graph2.add_edge('C','E',1)
graph2.add_edge('D','E',2)
graph2.add_edge('D','F',1)
graph2.add_edge('B','F',6)
graph2.add_edge('B','G',1)
graph2.add_edge('E','H',1)
graph2.add_edge('F','H',1)
graph2.add_edge('G','H',3)
graph2.add_edge('B','J',2)
graph2.add_edge('J','F',2)
graph2.add_edge('E','I',2)
graph2.add_edge('I','F',1)
graph2.add_edge('K','F',6)
graph2.add_edge('L','H',5)
graph2.add_edge('A','L',3)
graph2.add_edge('B','M',1)
graph2.add_edge('M','E',6)
graph2.add_edge('M','N',3)
graph2.add_edge('N','H',1)
graph2.add_edge('A','O',2)
graph2.add_edge('O','F',1)
graph2.add_edge('A','P',5)
graph2.add_edge('P','H',1)
graph2.add_edge('C','Q',3)
graph2.add_edge('Q','B',1)
graph2.add_edge('A','R',1)
graph2.add_edge('R','L',1)
graph2.add_edge('R','B',3)
graph2.add_edge('L','C',5)
graph2.add_edge('L','E',5)
graph2.add_edge('E','K',3)

# Now we will give more meaningful names to our nodes in order to use it for our business case
departure = 'A'
arrival = 'H'
inter1 = 'E'
inter2 = 'B'
inter3 = 'F'
intermediaries = [inter1,inter2,inter3]

find_optimal_intermediary(departure,arrival,intermediaries,graph2)

Without going through any intermediary points d = 3
Synthesis of the distance through each intermediary point:
	Through E, d = 3
	Through B, d = 6
	Through F, d = 4
The optimal intermediary point is E (increasing the distance between 'A' and 'H' of 0 distance units).


In [6]:
# In order to test the program, we create a graph as an example
graph3 = Graph()

# Then we add the different nodes
graph3.add_node('A') # (departure)
graph3.add_node('B')
graph3.add_node('C')
graph3.add_node('D')
graph3.add_node('E')
graph3.add_node('F')
graph3.add_node('G')
graph3.add_node('H')# (arrival)
graph3.add_node('I')
graph3.add_node('J')
graph3.add_node('K')
graph3.add_node('L')
graph3.add_node('M')
graph3.add_node('N')
graph3.add_node('O')
graph3.add_node('P')
graph3.add_node('Q')
graph3.add_node('R')
graph3.add_node('S')
graph3.add_node('T')
graph3.add_node('U')
graph3.add_node('V')

# Finally we create the different links between the edges
graph3.add_edge('A','C',1)
graph3.add_edge('A','B',2)
graph3.add_edge('C','D',2)
graph3.add_edge('D','B',3)
graph3.add_edge('C','E',1)
graph3.add_edge('D','E',2)
graph3.add_edge('D','F',1)
graph3.add_edge('B','F',6)
graph3.add_edge('B','G',1)
graph3.add_edge('E','H',1)
graph3.add_edge('F','H',1)
graph3.add_edge('G','H',3)
graph3.add_edge('B','J',2)
graph3.add_edge('J','F',2)
graph3.add_edge('E','I',2)
graph3.add_edge('I','F',1)
graph3.add_edge('K','F',6)
graph3.add_edge('L','H',5)
graph3.add_edge('A','L',3)
graph3.add_edge('B','M',1)
graph3.add_edge('M','E',6)
graph3.add_edge('M','N',3)
graph3.add_edge('N','H',1)
graph3.add_edge('A','O',2)
graph3.add_edge('O','F',1)
graph3.add_edge('A','P',5)
graph3.add_edge('P','H',1)
graph3.add_edge('C','Q',3)
graph3.add_edge('Q','B',1)
graph3.add_edge('A','R',1)
graph3.add_edge('R','L',1)
graph3.add_edge('R','B',3)
graph3.add_edge('L','C',5)
graph3.add_edge('L','E',5)
graph3.add_edge('E','K',3)
graph3.add_edge('A','S',6)
graph3.add_edge('S','F',4)
graph3.add_edge('F','B',1)
graph3.add_edge('S','H',5)
graph3.add_edge('B','T',1)
graph3.add_edge('T','K',7)
graph3.add_edge('C','U',3)
graph3.add_edge('U','D',1)
graph3.add_edge('U','J',7)
graph3.add_edge('O','V',3)
graph3.add_edge('V','U',4)
graph3.add_edge('V','B',4)

# Now we will give more meaningful names to our nodes in order to use it for our business case
departure = 'A'
arrival = 'H'
inter1 = 'E'
inter2 = 'B'
inter3 = 'F'
intermediaries = [inter1,inter2,inter3]

find_optimal_intermediary(departure,arrival,intermediaries,graph3)

Without going through any intermediary points d = 3
Synthesis of the distance through each intermediary point:
	Through E, d = 3
	Through B, d = 6
	Through F, d = 4
The optimal intermediary point is E (increasing the distance between 'A' and 'H' of 0 distance units).


Here we can observe the running times and big-O plots.

In [7]:
# running time of the algorithm when the graph input is "graph" with 8 nodes
import time
start_time = time.time()
find_optimal_intermediary(departure,arrival,intermediaries,graph)
end_time = time.time()
print("--- {} seconds ---".format(end_time-start_time))

Without going through any intermediary points d = 3
Synthesis of the distance through each intermediary point:
	Through E, d = 3
	Through B, d = 6
	Through F, d = 5
The optimal intermediary point is E (increasing the distance between 'A' and 'H' of 0 distance units).
--- 0.0011150836944580078 seconds ---


In [8]:
# running time of the algorithm when the graph input is "graph1" with 14 nodes
import time
start_time = time.time()
find_optimal_intermediary(departure,arrival,intermediaries,graph1)
end_time = time.time()
print("--- {} seconds ---".format(end_time-start_time))

Without going through any intermediary points d = 3
Synthesis of the distance through each intermediary point:
	Through E, d = 3
	Through B, d = 6
	Through F, d = 5
The optimal intermediary point is E (increasing the distance between 'A' and 'H' of 0 distance units).
--- 0.0011723041534423828 seconds ---


In [9]:
# running time of the algorithm when the graph input is "graph" with 18 nodes
import time
start_time = time.time()
find_optimal_intermediary(departure,arrival,intermediaries,graph2)
end_time = time.time()
print("--- {} seconds ---".format(end_time-start_time))

Without going through any intermediary points d = 3
Synthesis of the distance through each intermediary point:
	Through E, d = 3
	Through B, d = 6
	Through F, d = 4
The optimal intermediary point is E (increasing the distance between 'A' and 'H' of 0 distance units).
--- 0.001554250717163086 seconds ---


In [10]:
# running time of the algorithm when the graph input is "graph" with 22 nodes
import time
start_time = time.time()
find_optimal_intermediary(departure,arrival,intermediaries,graph3)
end_time = time.time()
print("--- {} seconds ---".format(end_time-start_time))

Without going through any intermediary points d = 3
Synthesis of the distance through each intermediary point:
	Through E, d = 3
	Through B, d = 6
	Through F, d = 4
The optimal intermediary point is E (increasing the distance between 'A' and 'H' of 0 distance units).
--- 0.003374338150024414 seconds ---


In [ ]:
# Plot returning running time for every generated graph.
%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use('ggplot')

plt.figure(figsize=(6,4))  

times = []
graphs = [graph,graph1,graph2]

n_nodes = [len(list(g.edges)) for g in graphs]


for g in graphs:
    start_time = time.time()
    find_optimal_intermediary(departure,arrival,intermediaries,g)
    end_time = time.time()
    times.append(end_time-start_time)
    
plt.xlabel('length of list (n)')
plt.ylabel('run time (secs)')
plt.title('Run Time Big(O) for combinations()')
plt.plot(n_nodes, times, 'o-');

In [ ]:
# Plot returning running time for every generated graph.
%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use('ggplot')

plt.figure(figsize=(6,4))  

times = []
graphs = [graph,graph1,graph2,graph3]

n_nodes = [len(list(g.edges)) for g in graphs]


for g in graphs:
    start_time = time.time()
    find_optimal_intermediary(departure,arrival,intermediaries,g)
    end_time = time.time()
    times.append(end_time-start_time)
    
plt.xlabel('length of list (n)')
plt.ylabel('run time (secs)')
plt.title('Run Time Big(O) for combinations()')
plt.plot(n_nodes, times, 'o-');


In [ ]:
n_nodes